<a href="https://colab.research.google.com/github/mbahadirk/OBSS-Competition/blob/main/OBSS_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

import os
import zipfile

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c obss-intern-competition-2025


Saving kaggle.json to kaggle.json
 93% 923M/995M [00:00<00:00, 1.35GB/s]
100% 995M/995M [00:00<00:00, 1.35GB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from datetime import datetime

save_dir = "/content/drive/MyDrive/git_checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
!pip install -q transformers torchvision sentencepiece datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import torch
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import pickle

In [ ]:
with zipfile.ZipFile("obss-intern-competition-2025.zip", "r") as zip_ref:
    zip_ref.extractall("obss_caption_data")

In [ ]:
df = pd.read_csv("/content/obss_caption_data/train.csv")
print(f"Toplam veri: {len(df)}")

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

Toplam veri: 21367


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
torch.manual_seed(42)

In [ ]:
from transformers import GitProcessor, GitForCausalLM

processor = GitProcessor.from_pretrained("microsoft/git-base")
model = GitForCausalLM.from_pretrained("microsoft/git-base").to(device)


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
class CaptionDataset(Dataset):
    def __init__(self, dataframe, processor, image_dir):
        self.dataframe = dataframe.reset_index(drop=True)
        self.processor = processor
        self.image_dir = image_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.image_dir, f"{row['image_id']}.jpg")
        image = Image.open(image_path).convert("RGB")
        caption = row['caption']

        inputs = self.processor(images=image, text=caption, padding="max_length", truncation=True, return_tensors="pt")
        inputs["labels"] = inputs.input_ids.clone()
        for k in inputs:
            inputs[k] = inputs[k].squeeze()
        return inputs


In [ ]:
train_dataset = CaptionDataset(train_df, processor, "obss_caption_data/train/train")
val_dataset = CaptionDataset(val_df, processor, "obss_caption_data/train/train")

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, pin_memory=True)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scaler = torch.cuda.amp.GradScaler()

<ipython-input-21-cb0dbc491eee>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
num_epochs = 30
patience = 3
min_val_loss = float('inf')
early_stopping = 0
tracking_info = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        input_ids = batch['input_ids'].to(device)
        pixel_values = batch['pixel_values'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            pixel_values = batch['pixel_values'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with torch.cuda.amp.autocast():
                outputs = model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
            val_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    tracking_info.append((avg_train_loss, avg_val_loss, optimizer.param_groups[0]['lr']))

    print(f"\nEpoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f}")

    model_filename = f"epoch_{epoch+1:02d}_val{avg_val_loss:.4f}.pt"
    model_path = os.path.join(save_dir, model_filename)
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to: {model_path}")

    if avg_val_loss < min_val_loss:
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pt"))
        print("🎯 Best model updated.\n")
        min_val_loss = avg_val_loss
        early_stopping = 0
    else:
        early_stopping += 1
        if early_stopping >= patience:
            print("⛔ Early stopping devrede!")
            break

    scheduler.step()


Epoch 1 - Training:   0%|          | 0/1202 [00:00<?, ?it/s]<ipython-input-22-a475e107b020>:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating:   0%|          | 0/134 [00:00<?, ?it/s]<ipython-input-22-a475e107b020>:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating: 100%|██████████| 134/134 [00:23<00:00,  5.76it/s]



Epoch 1: Train Loss = 2.1243 | Val Loss = 0.2900
Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_01_val0.2900.pt
🎯 Best model updated.



Validating: 100%|██████████| 134/134 [00:23<00:00,  5.77it/s]



Epoch 2: Train Loss = 0.2324 | Val Loss = 0.2023
Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_02_val0.2023.pt
🎯 Best model updated.



Validating: 100%|██████████| 134/134 [00:23<00:00,  5.82it/s]


...


Epoch 7: Train Loss = 0.1294 | Val Loss = 0.1781
Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_07_val0.1781.pt


Validating: 100%|██████████| 134/134 [00:23<00:00,  5.72it/s]



Epoch 8: Train Loss = 0.1216 | Val Loss = 0.1795
Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_08_val0.1795.pt


Validating: 100%|██████████| 134/134 [00:23<00:00,  5.74it/s]



Epoch 9: Train Loss = 0.1149 | Val Loss = 0.1814
Model saved to: /content/drive/MyDrive/blip_checkpoints/epoch_09_val0.1814.pt
⛔ Early stopping devrede!


In [ ]:
import pickle
with open(os.path.join(save_dir, "/content/training_log.pkl"), "wb") as f:
    pickle.dump(tracking_info, f)